In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from copy import deepcopy

In [15]:
# Perform updates on the python modules
# !pip install --user -q opencv-python
# !pip install --user -q numpy
# !pip install --user -q matplotlib

In [16]:
# version check, see if there are newer versions
print(cv2.__version__)
print(np.__version__)
print(mpl.__version__)

4.2.0
1.18.2
3.1.1


In [17]:
# make sure the current directory is also where the folder the notebook resides in!
import os
os.getcwd()

'C:\\Users\\gdioni\\Desktop\\Jupyter Notebook\\OpenCV'

# Image Processing Events

## Tracking

In [18]:
def Set_Tracker(t_type_lst: list, t_type_index : int):
  # get the tracker list and get the current index
  tracker_name = t_type_lst[t_type_index]
  # Tracker creation
  tracker = None
  if tracker_name == t_type_lst[0]: 
    # BOOSTING
    tracker = cv2.TrackerBoosting_create()
  elif tracker_name == t_type_lst[1]:
    # MIL
    tracker = cv2.TrackerMIL_create()
  elif tracker_name == t_type_lst[2]:
    # KCF
    tracker = cv2.TrackerKCF_create()
  elif tracker_name == t_type_lst[3]:
    # TLD
    tracker = cv2.TrackerTLD_create()
  elif tracker_name == t_type_lst[4]:
    # MEDIANFLOW
    tracker = cv2.TrackerMedianFlow_create()
  elif tracker_name == t_type_lst[5]:
    # GOTURN
    tracker = cv2.TrackerGOTURN_create()
  elif tracker_name == t_type_lst[6]:
    # CSRT
    tracker = cv2.TrackerCSRT_create()
  elif tracker_name == t_type_lst[7]:
    tracker = cv2.TrackerMOSSE_create()
  else:
    print('Something went wrong initializing the tracker')
  return tracker
def Track_Object(dframe : np.array, tracker : 'tracker')->np.array:
  global ball_tracker_bbox, thickness, ball_bbox_upscale, downscale_size

  tracked, ball_tracker_bbox = tracker.update(dframe)
  FLAG_TRACKER_OK = tracked
  if not tracked:
    ball_tracker_bbox =  (-1,-1,-1,-1)
  return dframe

## Detection using YOLOv3

### Detection Helper Methods

In [19]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Draw the predicted bounding box
def drawPred(frame, classId, conf, left, top, width, height):
    global bbox_detector_color, thickness

    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (left+width, top+height), bbox_detector_color, thickness, cv2.LINE_AA)
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)
        
    #Display the label at the top of the bounding box
    # labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    # top = max(top, labelSize[1])
    # cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    # cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)
    return

# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs):
    global ball_detector_bbox, FLAG_DETECT_OK
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            if detection[4] > objectnessThreshold :
                scores = detection[5:]
                classId = np.argmax(scores)
                confidence = scores[classId]
                if confidence > confThreshold and classId == 32:
                    # 'classId == 32' are sports ball objects
                    # only interested in detected sports ball objects
                    center_x = int(detection[0] * frameWidth)
                    center_y = int(detection[1] * frameHeight)
                    width = int(detection[2] * frameWidth)
                    height = int(detection[3] * frameHeight)
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)
                    classIds.append(classId)
                    confidences.append(float(confidence))
                    boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
#     print('number of indices: ' + str(len(indices)))
    
    if len(indices) <= 0:
      ball_boxes_bb = (-1,-1,-1,-1)
      FLAG_DETECT_OK = False
      return
    # bounding boxes for detection
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        
        # get the bounding box
        ball_detector_bbox = (left, top, width, height)
    FLAG_DETECT_OK = True
    return
    
def Detect_Object(frame: np.array):
  # Create a 4D blob from a frame.
  blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

  # Sets the input to the network
  net.setInput(blob)

  # Runs the forward pass to get output of the output layers
  outs = net.forward(getOutputsNames(net))
  #outs = net.forward()

  # Remove the bounding boxes with low confidence
  postprocess(frame, outs)
  return frame

# Keyboard Events

In [20]:
def EventKeyR_iniReset():
  global ini_frame_copy, ini_frame
  ini_frame = np.copy(ini_frame_copy)
  ini_flag = False
  return
def EventKeyEnter_ini():
  global ini_flag
  ini_flag = True
  return

# Trackbar Events

In [21]:
def EventTrkChangeTrackerType(*args):
  '''
  Changes t_type_index to the value from the trackbar 
  '''
  global t_type_index, FLAG_CHANGE_TRACKER
  t_type_index = args[0]
  FLAG_CHANGE_TRACKER = True
  return

# HELPER METHOD

In [22]:
def ValidateTracker(tracker_bb:tuple, detector_bb:tuple, tracker: 'Tracker object', frame) -> tuple:
  '''
  If the tracker's center point goes beyond the detector's rectangle
  set the tracker to the detector's bounding box
  '''
  global t_type_index, t_type_lst
  print('tracker: ' + str(tracker_bb))
  print('detector: ' + str(detector_bb))
  t0, t1, tw, th = tracker_bb
  d0, d1, dw, dh = detector_bb
  
  if sum([d0, d1, dw, dh]) <= 0:
    return tracker, tracker_bb
#   t0, t1, tw, th = tracker_bb
#   d0, d1, dw, dh = detector_bb

  # get tracker center
  x, y = t0+(tw//2), t1+(th//2)

  # get detector left, top, right, bottom coordinates
  x1, y1, x2, y2 = d0, d1, d0+dw, d1+dh
  valid = PointInsideRectange(x1,y1,x2,y2, x, y)
  row, col, dim = frame.shape
  d0, d1, dw, dh = int(max(0, d0)), int(max(0, d1)), int(max(0, dw)), int(max(0, dh))
  if not valid and (0 <= d0 and 0 <= dw and d0 + dw <= col and 0 <= d1 and 0 <= dh and d1 + dh <= row):
    '''
    if the tracker went outside the detector rectangle and detector_bb won't cause initialization issues
    '''
    print(d0 > 0, d1 > 0, d0+dw > 0, d1+dh > 0)
    tracker = Set_Tracker(t_type_lst, t_type_index)
    _ = tracker.init(frame_downscale, (d0, d1, dw, dh))
  elif not valid:
    '''
    if the tracker was invalid but the detector_bb will cause an issue
    simply invalidate the current tracker_bb and wait later
    '''
    tracker_bb = (-1,-1,-1,-1)
  return tracker, tracker_bb


def PointInsideRectange(x1, y1, x2, y2, x, y) -> bool:
  if x > x1 and x < x2 and y > y1 and y < y2:
    return True
  else:
    return False

# Main Method

In [24]:
# insert the video path here for ball tracking
path_vid = '.\soccer-ball.mp4'
cap = cv2.VideoCapture(path_vid)

'''
+++ TRACKER configuration
'''
win_name = 'Tracker'
ini_win_name = 'Select object bounding box'
t_type_lst = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'CSRT', 'MOSSE']
t_type_index = 0 # used to change tracker type
t_type_text = t_type_lst[t_type_index]
t_type_min, t_type_max = 0, len(t_type_lst)-1
txt_font_size, txt_thickness = 2, 3

k = 0 # used for key press
k_timer = 1 # wait time for keyboard press
frame_counter = 0 # used for resetting the frame

cv2.namedWindow(win_name, cv2.WINDOW_AUTOSIZE)

# Bounding box
ball_detector_bbox = (-1, -1, -1, -1) # bounding box of Detector
ball_tracker_bbox = (-1, -1, -1, -1) # bounding box of Tracker

bbox_tracker_color = (255, 125, 0)
bbox_detector_color = (125, 255, 0)
FLAG_CHANGE_TRACKER = False
FLAG_TRACKER_OK = False
FLAG_DETECT_OK = False # Flag to use if ball was detected
detection_timer = 10 # Detect every Nth frame
thickness = 2
FLAG_INIT = False
downscale_size = 0.75
tracker = None
no_detect = 0

'''
--- TRACKER configurations
'''

'''
+++ YOLO v3 detector variables
'''
# Initialize the parameters
objectnessThreshold = 0.5 # Objectness threshold
confThreshold = 0.5       # Confidence threshold
nmsThreshold = 0.4        # Non-maximum suppression threshold
inpWidth = 416            # Width of network's input image
inpHeight = 416           # Height of network's input image

y3_weights_path = './data/yolov3.weights'
y3_config_path = './data/yolov3.cfg'
coco_classes_path = './data/coco.names' # file with multiple classes
ball_classes_path = './data/target.names' # only sports ball
choice_path = coco_classes_path
classes = None
with open(choice_path, 'rt') as f:
  classes = f.read().strip('\n').split('\n')

# load the network
net = cv2.dnn.readNetFromDarknet(y3_config_path, y3_weights_path)
# net = cv2.dnn.DNN_TARGET_OPENCL(y3_config_path, y3_weights_path)

'''
--- YOLO v3 Configuration
'''

if not cap.isOpened():
  print('Could not open the video')
else:
  cv2.createTrackbar('Trackers: ', win_name, t_type_min, t_type_max, EventTrkChangeTrackerType)
  while cap.isOpened() and k != 27:

    if k == ord('r'):
      '''
      When the 'r' keyboard button is pressed, reset the video
      '''
      # reset frame to position 0
      frame_counter = 0
      cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter)

    # read the frame  
    frameIsRead, f = cap.read()

    if not frameIsRead:
      print('Video read unsuccessful')
      break

    # keep track of frame tick
    frame_counter += 1
    # Attempt to down scale for FPS Boost
    row, col, dim = f.shape
    row, col = int(row*downscale_size), int(col*downscale_size)
    dim = (col, row)
    frame_downscale = cv2.resize(f, dim, interpolation = cv2.INTER_AREA)
    if not FLAG_INIT:
      ball_tracker_bbox = cv2.selectROI(ini_win_name, frame_downscale)
      tracker = Set_Tracker(t_type_lst, t_type_index)
      FLAG_INIT = tracker.init(frame_downscale, ball_tracker_bbox)
      if FLAG_INIT:
        print('Tracker Initialized')
      else:
        print('Tracker not initialized')
      cv2.destroyWindow(ini_win_name)
    
    # perform detection every Nth frames
    ball_detector_bbox = (-1, -1, -1, -1)
    if (frame_counter - 1) % detection_timer == 0:
      # ball_detector_bbox gets updated inside Detect_Object
      detect_frame = Detect_Object(frame_downscale)
      if not FLAG_DETECT_OK:
        no_detect += 1
      else:
        no_detect = 0

    # ball_tracker_bbox gets updated inside Track_Objects
    track_frame = Track_Object(frame_downscale, tracker)

    tracker, ball_tracker_bbox = ValidateTracker(ball_tracker_bbox, ball_detector_bbox, tracker, frame_downscale)
    t0, t1, t2, t3 = ball_tracker_bbox
    d0, d1, d2, d3 = ball_detector_bbox

    print('Detector: ' + str(sum(ball_detector_bbox)))
    print('Tracker: ' + str(sum(ball_tracker_bbox)))
    print('\n')
    if sum(ball_detector_bbox) >= 0:
      cv2.rectangle(frame_downscale, (int(d0), int(d1)), (int(d0+d2), int(d1+d3)), bbox_detector_color, thickness, cv2.LINE_AA)
    if sum(ball_tracker_bbox) >= 0 and no_detect < 2:
      cv2.rectangle(frame_downscale, (int(t0), int(t1)), (int(t0+t2), int(t1+t3)), bbox_tracker_color, thickness, cv2.LINE_AA)
      cv2.putText(frame_downscale, 'Tracker: ' + str(tracker),(10,row), cv2.FONT_HERSHEY_PLAIN, txt_font_size, (255, 255, 255), txt_thickness, cv2.LINE_AA)
    else:
      cv2.putText(frame_downscale, 'No object tracked',(10,row), cv2.FONT_HERSHEY_PLAIN, txt_font_size, (255, 255, 255), txt_thickness, cv2.LINE_AA)
    cv2.imshow(win_name, frame_downscale)
    
    # read keyboard input
    k = cv2.waitKey(k_timer)

# release of the cap object
cap.release()
# close all windows
cv2.destroyAllWindows()

Tracker Initialized
tracker: (407.0, 225.0, 105.0, 98.0)
detector: (427, 242, 80, 75)
Detector: 824
Tracker: 835.0


tracker: (407.0, 222.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 832.0


tracker: (406.0, 222.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 831.0


tracker: (406.0, 221.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 830.0


tracker: (405.0, 223.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 831.0


tracker: (402.0, 224.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 829.0


tracker: (401.0, 224.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 828.0


tracker: (397.0, 223.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 823.0


tracker: (398.0, 224.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 825.0


tracker: (395.0, 224.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 822.0


tracker: (395.0, 227.0, 105.0, 98.0)


tracker: (737.0, 295.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1235.0


tracker: (695.0, 255.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1153.0


tracker: (729.0, 259.0, 105.0, 98.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1191.0


tracker: (750.0, 246.0, 105.0, 98.0)
detector: (107, 224, 115, 105)
True True True True
Detector: 551
Tracker: 1199.0


tracker: (91.0, 224.0, 115.0, 105.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 535.0


tracker: (76.0, 222.0, 115.0, 105.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 518.0


tracker: (60.0, 217.0, 115.0, 105.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 497.0


tracker: (45.0, 213.0, 115.0, 105.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 478.0


tracker: (32.0, 212.0, 115.0, 105.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 464.0


tracker: (20.0, 210.0, 115.0, 105.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 450.0


tracker: (9.0, 209.0, 115.0, 10

tracker: (445.0, 420.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1012.0


tracker: (459.0, 423.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1029.0


tracker: (452.0, 428.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1027.0


tracker: (456.0, 421.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1024.0


tracker: (459.0, 423.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1029.0


tracker: (468.0, 410.0, 37.0, 110.0)
detector: (401, 380, 140, 124)
Detector: 1045
Tracker: 1025.0


tracker: (478.0, 408.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1033.0


tracker: (476.0, 401.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1024.0


tracker: (477.0, 398.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1022.0


tracker: (491.0, 382.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1020.0


tracker: (480.0, 388.0, 37.0, 110.0)
detecto

tracker: (225.0, 0.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 372.0


tracker: (210.0, 0.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 357.0


tracker: (217.0, 56.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 420.0


tracker: (205.0, 19.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 371.0


tracker: (211.0, 64.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 422.0


tracker: (196.0, 20.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 363.0


tracker: (205.0, 17.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 369.0


tracker: (211.0, 26.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 384.0


tracker: (214.0, 47.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 408.0


tracker: (214.0, 63.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 424.0


tracker: (222.0, 98.0, 37.0, 110.0)
detector: (-1, -1, -1, -1)
Detector:

tracker: (683.0, 163.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1173.0


tracker: (686.0, 151.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1164.0


tracker: (689.0, 137.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1153.0


tracker: (692.0, 125.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1144.0


tracker: (693.0, 115.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1135.0


tracker: (696.0, 109.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1132.0


tracker: (697.0, 103.0, 162.0, 165.0)
detector: (709, 119, 146, 146)
Detector: 1120
Tracker: 1127.0


tracker: (698.0, 97.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1122.0


tracker: (695.0, 79.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1101.0


tracker: (694.0, 69.0, 162.0, 165.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 1090.0


tracker: (693.0, 59.0, 162.0, 165.0)


tracker: (555.0, 146.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 905.0


tracker: (562.0, 146.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 912.0


tracker: (568.0, 146.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 918.0


tracker: (574.0, 147.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 925.0


tracker: (579.0, 151.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 934.0


tracker: (586.0, 154.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 944.0


tracker: (589.0, 158.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 951.0


tracker: (594.0, 163.0, 79.0, 125.0)
detector: (561, 170, 110, 110)
Detector: 951
Tracker: 961.0


tracker: (596.0, 166.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 966.0


tracker: (597.0, 168.0, 79.0, 125.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 969.0


tracker: (596.0, 171.0, 79.0, 125.0)
detector: (-1, -1,

tracker: (400.0, 235.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 828.0


tracker: (403.0, 235.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 831.0


tracker: (406.0, 233.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 832.0


tracker: (407.0, 233.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 833.0


tracker: (389.0, 237.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 819.0


tracker: (370.0, 242.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 805.0


tracker: (353.0, 244.0, 98.0, 95.0)
detector: (358, 247, 100, 92)
Detector: 797
Tracker: 790.0


tracker: (339.0, 242.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 774.0


tracker: (323.0, 242.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 758.0


tracker: (308.0, 242.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 743.0


tracker: (291.0, 244.0, 98.0, 95.0)
detector: (-1, -1, -1, -1)
Det

tracker: (247.0, 170.0, 94.0, 80.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 591.0


tracker: (247.0, 169.0, 94.0, 80.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 590.0


tracker: (245.0, 168.0, 94.0, 80.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 587.0


tracker: (245.0, 164.0, 94.0, 80.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 583.0


tracker: (243.0, 159.0, 94.0, 80.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 576.0


tracker: (242.0, 157.0, 94.0, 80.0)
detector: (-1, -1, -1, -1)
Detector: -4
Tracker: 573.0


tracker: (242.0, 156.0, 94.0, 80.0)
detector: (255, 166, 71, 70)
Detector: 562
Tracker: 572.0


